In [1]:
import gdax
%matplotlib inline
public_client = gdax.PublicClient()

In [10]:
class SMA:
    def __init__(self, window):
        self.window = []
        self.full_len = window
        self.count = 0
        self.index = 0
        self.avg = 0.
        
    def tick(self, val):
        ret = None
        if self.count == self.full_len:
            self.avg -= self.window[self.index]
            self.avg += val
            self.window[self.index] = val
            self.index += 1
            self.index = self.index % self.full_len
            ret = self.avg / self.full_len
        else:
            self.window.append(val)
            self.avg += val
            self.index += 1
            self.index = self.index % self.full_len
            self.count += 1
            if self.count == self.full_len:
                ret = self.avg / self.count
        return ret
    

In [12]:
five = SMA(5)
for i in range(10):
    print(five.tick(i))
        

0.0
0.5
1.0
1.5
2.0
3.0
4.0
5.0
6.0
7.0


In [61]:
class DoubleSMADecision:
    def __init__(self, slow=50, fast=5, cutoff=0.1):
        self.slow = SMA(slow)
        self.fast = SMA(fast)
        self.cutoff = cutoff
        
    def tick(self, price):
        ret = None
        slow = self.slow.tick(price)
        fast = self.fast.tick(price)
        if (slow != None and fast != None):
            ret = (fast - slow) > self.cutoff
            #print("Slow: {}  Fast: {} Diff: {}".format(slow, fast, (fast - slow)))
        return ret

In [31]:
dsd = DoubleSMADecision()
for i in range(10):
    print(dsd.tick(i + 1))

False
False
False
False
False
False
True
True
True
True


In [84]:
class Trader:
    def __init__(self, algo, fee=0.0025):
        self.algo = algo
        self.value = 100.
        self.stance = False
        self.last_price = 0
        self.num_trades = 0
        self.fee = fee
        
    def tick(self, price):
        if self.last_price == 0:
            self.last_price = price
        decision = self.algo.tick(price)
        if decision != self.stance:
            self.stance = decision
            self.value -= (self.value * self.fee)
            self.num_trades += 1
        elif decision:
            self.value += self.value * ((price - self.last_price) / self.last_price)
        self.last_price = price
        return self.value
                

In [82]:
import gdax
import numpy as np

%matplotlib inline

public_client = gdax.PublicClient()

# Save historical data, take number of points and granularity
from datetime import datetime, timedelta
from math import ceil
from time import sleep

def historic_rates(product, num_candles, granularity=60, max_per=401, initial=None):
    start = initial if initial else datetime.utcnow().replace(microsecond=0)
    delta = timedelta(seconds=granularity) * max_per
    full = ceil(num_candles / max_per)
    rates = []
    for i in range(1, full + 1):
        frm = start - (delta * i)
        to  = frm + delta
        print('From: {}, To: {}'.format(frm.isoformat(), to.isoformat()))
        rates.extend(public_client.get_product_historic_rates(product, start=frm.isoformat(), end=to.isoformat(), granularity=granularity))
        sleep(0.33)
    return rates

In [83]:
rates = historic_rates('ETH-USD', 10 * 10000)
prices = [x[4] for x in rates]
prices.reverse()

From: 2017-12-30T14:39:13, To: 2017-12-30T21:20:13
From: 2017-12-30T07:58:13, To: 2017-12-30T14:39:13
From: 2017-12-30T01:17:13, To: 2017-12-30T07:58:13
From: 2017-12-29T18:36:13, To: 2017-12-30T01:17:13
From: 2017-12-29T11:55:13, To: 2017-12-29T18:36:13
From: 2017-12-29T05:14:13, To: 2017-12-29T11:55:13
From: 2017-12-28T22:33:13, To: 2017-12-29T05:14:13
From: 2017-12-28T15:52:13, To: 2017-12-28T22:33:13
From: 2017-12-28T09:11:13, To: 2017-12-28T15:52:13
From: 2017-12-28T02:30:13, To: 2017-12-28T09:11:13
From: 2017-12-27T19:49:13, To: 2017-12-28T02:30:13
From: 2017-12-27T13:08:13, To: 2017-12-27T19:49:13
From: 2017-12-27T06:27:13, To: 2017-12-27T13:08:13
From: 2017-12-26T23:46:13, To: 2017-12-27T06:27:13
From: 2017-12-26T17:05:13, To: 2017-12-26T23:46:13
From: 2017-12-26T10:24:13, To: 2017-12-26T17:05:13
From: 2017-12-26T03:43:13, To: 2017-12-26T10:24:13
From: 2017-12-25T21:02:13, To: 2017-12-26T03:43:13
From: 2017-12-25T14:21:13, To: 2017-12-25T21:02:13
From: 2017-12-25T07:40:13, To: 

From: 2017-11-15T18:38:13, To: 2017-11-16T01:19:13
From: 2017-11-15T11:57:13, To: 2017-11-15T18:38:13
From: 2017-11-15T05:16:13, To: 2017-11-15T11:57:13
From: 2017-11-14T22:35:13, To: 2017-11-15T05:16:13
From: 2017-11-14T15:54:13, To: 2017-11-14T22:35:13
From: 2017-11-14T09:13:13, To: 2017-11-14T15:54:13
From: 2017-11-14T02:32:13, To: 2017-11-14T09:13:13
From: 2017-11-13T19:51:13, To: 2017-11-14T02:32:13
From: 2017-11-13T13:10:13, To: 2017-11-13T19:51:13
From: 2017-11-13T06:29:13, To: 2017-11-13T13:10:13
From: 2017-11-12T23:48:13, To: 2017-11-13T06:29:13
From: 2017-11-12T17:07:13, To: 2017-11-12T23:48:13
From: 2017-11-12T10:26:13, To: 2017-11-12T17:07:13
From: 2017-11-12T03:45:13, To: 2017-11-12T10:26:13
From: 2017-11-11T21:04:13, To: 2017-11-12T03:45:13
From: 2017-11-11T14:23:13, To: 2017-11-11T21:04:13
From: 2017-11-11T07:42:13, To: 2017-11-11T14:23:13
From: 2017-11-11T01:01:13, To: 2017-11-11T07:42:13
From: 2017-11-10T18:20:13, To: 2017-11-11T01:01:13
From: 2017-11-10T11:39:13, To: 

In [89]:
for val in np.arange(1, 1000, 10):
    trader = Trader(DoubleSMADecision(slow=10 * val, fast=val, cutoff=0.2))
    final = 0
    for price in prices:
        final = trader.tick(price)
    print("{}: {} ({})".format(val, final, trader.num_trades))

1: 2.0565747877211875e-05 (11420)
11: 12.714614838505256 (1490)
21: 28.22937499801399 (833)
31: 30.148875042018677 (629)
41: 49.65871226066786 (471)
51: 61.28915224732828 (373)
61: 86.76248234004477 (314)
71: 85.54091144199967 (276)
81: 118.09106602516313 (218)
91: 109.44476574311717 (194)
101: 125.85019078600193 (176)
111: 118.22645226022632 (156)
121: 125.49788068174234 (140)
131: 131.0615902471209 (132)
141: 131.60119741124092 (118)
151: 117.95895082630025 (112)
161: 125.44954513857758 (106)
171: 135.36545758494242 (110)
181: 152.55955578405715 (94)
191: 152.4710706672674 (90)
201: 144.99476521736344 (90)
211: 133.0181245194908 (86)
221: 140.02600233803392 (84)
231: 140.52111213036963 (82)
241: 132.58803242751154 (80)
251: 131.0680555156313 (80)
261: 130.8758871933612 (70)
271: 123.37048005090797 (74)
281: 131.59164519470804 (66)
291: 134.85401620953687 (68)
301: 134.36874042507543 (62)
311: 126.49580962563421 (60)
321: 120.37582337095003 (62)
331: 128.27545645050247 (56)
341: 123.1

In [68]:
print(range(0.1, 2.0, 0.1))

TypeError: 'float' object cannot be interpreted as an integer